In [1]:
# Pkg.add( "CUDAdrv")
# Pkg.add("CUDAnative")
# Pkg.add("CuArrays")

In [2]:
#Pkg.add("StatsBase")

In [2]:
using StatsBase: wsample


In [3]:
cd(@__DIR__)

In [4]:
#Pkg.add("Flux")

In [5]:
#Pkg.test("Flux")

In [1]:
using Flux
using CuArrays

In [4]:
shakespear = read("100-0.txt", String)
print("B")

B

In [5]:
train = shakespear[3000:end]
length(train)

5793398

In [6]:
text = collect(String(train))
length(text)

5793398

In [7]:
text = replace(text, '$' => "")
text = replace(text, '&' => "")
text = replace(text, '@' => "")
text = replace(text, '*' => "")
text = replace(text, 'æ' => "")
text = replace(text, 'à' => "")
text = replace(text, 'œ' => "")
text = replace(text, 'Æ' => "")
text = replace(text, 'è' => "")
text = replace(text, '%' => "")
text = replace(text, '\\' => "")
text = replace(text, '/'=> "")
text = replace(text, '\'=> "")
text = replace(text, '_' => "")
text = replace(text, '|' => "")
text = replace(text, '[' => "")
text = replace(text, ']'=> "")
text = replace(text, '{' => "")
text = replace(text, '}' => "")
text = replace(text, '`' => "")

text = replace(text, '0' => "")
text = replace(text, '9' => "")
text = replace(text, '8' => "")
text = replace(text, '7' => "")
text = replace(text, '6' => "")
text = replace(text, '5' => "")
text = replace(text, '4' => "")
text = replace(text, '3' => "")
text = replace(text, '2' => "")
text = replace(text, '1' => "")

LoadError: syntax: invalid character literal

In [8]:
alphabet = [unique(text)..., '_']
print(length(alphabet), alphabet)

90Any['\n', '\r', ' ', '1', 'F', 'r', 'o', 'm', 'f', 'a', 'i', 'e', 's', 't', 'c', 'u', 'w', 'd', 'n', ',', 'T', 'h', 'b', 'y', '’', 'g', 'v', 'B', 'p', 'l', 'H', ':', '-', 'M', 'k', 'A', 'W', 'P', '.', '2', 'z', ';', 'I', '‘', 'S', 'x', '3', 'L', 'N', 'D', '?', 'O', 'C', '4', 'U', 'q', '5', '6', '7', 'j', 'R', 'Y', '(', ')', '8', '9', '0', 'G', 'V', 'E', 'K', '!', 'J', "", '—', '“', '”', '[', '_', ']', 'Q', '"', 'X', '\'', '|', 'Z', '`', '\t', '}', '_']

In [9]:
using Flux:onehot
text = map(ch -> onehot(ch, alphabet), text)

5793398-element Array{Flux.OneHotVector,1}:
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, true, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, fals

In [10]:
stop = onehot('_', alphabet)

90-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [11]:
N = length(alphabet)
seqlen = 50
nbatch = 50


50

In [12]:
N

90

In [13]:
using Flux: batchseq, chunk
using Base.Iterators: partition
Xs = collect(partition(batchseq(chunk(text,nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch),stop), seqlen))


2318-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 [[false true … false false; true false … false false; … ; false false … false false; false false … false false], [true false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false false … false true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false 

In [14]:
m = Chain(
    LSTM(N,128),
    LSTM(128,128),
    LSTM(128,128),
    Dense(128,N),
    softmax)

m = gpu(m)

Chain(Recur(LSTMCell(90, 128)), Recur(LSTMCell(128, 128)), Recur(LSTMCell(128, 128)), Dense(128, 90), NNlib.softmax)

In [15]:
using Flux: throttle, crossentropy

function loss(xs, ys)
    l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
    Flux.truncate!(m)
    return l
end

loss (generic function with 1 method)

In [16]:
?Flux.Optimise.ADAM

```
ADAM(params, η = 0.001; β1 = 0.9, β2 = 0.999, ϵ = 1e-08, decay = 0)
```

[ADAM](https://arxiv.org/abs/1412.6980v8) optimiser.


In [17]:
opt = Flux.Optimise.ADAM(params((m),0.01))

#43 (generic function with 1 method)

In [18]:
?Flux.train!

```
train!(loss, data, opt)
```

For each datapoint `d` in `data` computes the gradient of `loss(d...)` through backpropagation and calls the optimizer `opt`.

Takes a callback as keyword argument `cb`. For example, this will print "training" every 10 seconds:

```julia
Flux.train!(loss, data, opt,
            cb = throttle(() -> println("training"), 10))
```

The callback can return `:stop` to interrupt the training loop.

Multiple optimisers and callbacks can be passed to `opt` and `cb` as arrays.


In [19]:

tx, ty = (Xs[5],Ys[5])
evalcb = () -> @show loss(tx,ty)
data = zip(Xs, Ys)


Base.Iterators.Zip2{Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1},Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}}(Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}[[[true false … false false; false true … false false; … ; false false … false false; false false … false false], [false true … false false; true false … false false; … ; false false … false false; false false … false false], [true false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false false … false true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [fal

In [20]:
using Flux: @epochs
@epochs 2 Flux.train!(loss, data, opt, cb = throttle(evalcb,120))

┌ Info: Epoch 1
└ @ Main C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\optimise\train.jl:93


MethodError: MethodError: no method matching unbroadcast(::Tuple{Int64,Int64}, ::CuArray{Float32,2})
Closest candidates are:
  unbroadcast(!Matched::AbstractArray, ::Any) at C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\tracker\array.jl:334
  unbroadcast(!Matched::Number, ::Any) at C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\tracker\array.jl:339
  unbroadcast(!Matched::Base.RefValue{#s39} where #s39<:Function, ::Any) at C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\tracker\array.jl:340
  ...

In [21]:
function sample(m, alphabet, len; temp = 1)
  m = gpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println


VG
0m8q’fcuh_flNA"f6l-S9gTc]x5IN]`We.?-!pARbH””o`kx|‘CL2F5nwt'wHmU;.l 	3_eyr“pzxP|v}PqlcP_D, oJ8KF6gmg'_Y.L0GZALj]]u,vHkUz;FVWbDFn[5Wu)w—	h]wI	-vbsu36snlw 'I”"S1Q}_Z1p3  3qjA[j`f
:N::QZ)QrSAW-CxQ2w"—rY_u
Ei
RoqQ
0z	qv!!E’v4Rv':_"]p 	y4}Zey9LpN?ej9Uh7`P]xo”Rw8,f_`lh
N_,biM|TCC,_173aN2Eher}9xa1zWwbM!—lh’ PH”RXP Cx
UW8Mm-DT
8[BL}—6rN W)xOW,[X(3IhOM;
i.d:zYZKiKG;uGgmdjn_Y6zuuWv4zkx—eSpXq	C86`Q _zSZgR3
I
'7[Z!)UmZq!r
